In [1]:
import os
import time
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import cv2

In [2]:
IMG_H = 64
IMG_W = 64
IMG_C = 3

In [3]:
def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [4]:
def load_image(image_path):
  img = tf.io.read_file(image_path)
  img = tf.io.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, [64, 64])
  img = tf.cast(img, tf.float32)
  img = (img - 127.5) / 127.5

  return img

In [5]:
def tf_dataset(images_path, batch_size):
  ds = tf.data.Dataset.from_tensor_slices(images_path)
  ds = ds.shuffle(buffer_size=1000).map(load_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)
  ds = ds.batch(batch_size).prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

  return ds

In [6]:
def build_generator(latent_dim):
    noise = L.Input((latent_dim), name="noise_input")

    ## 1.
    x = L.Dense(256)(noise)
    x = L.LeakyReLU(0.2)(x)

    ## 2.
    x = L.Dense(1024)(x)
    x = L.LeakyReLU(0.2)(x)

    ## 2.
    x = L.Dense(4096)(x)
    x = L.LeakyReLU(0.2)(x)

    ## 4.
    x = L.Dense(IMG_H * IMG_W * IMG_C)(x)
    x = L.LeakyReLU(0.2)(x)

    ## 5.
    x = L.Reshape((IMG_H, IMG_W, IMG_C))(x)

    fake_output = L.Activation("tanh")(x)

    return Model(noise, fake_output, name="generator")

In [7]:
def build_discriminator():
    inputs = L.Input((IMG_H, IMG_W, IMG_C), name="disc_input")

    ## 1.
    x = L.Flatten()(inputs)

    ## 2.
    x = L.Dense(4096)(x)
    x = L.LeakyReLU(0.2)(x)
    x = L.Dropout(0.3)(x)

    ## 3.
    x = L.Dense(1024)(x)
    x = L.LeakyReLU(0.2)(x)
    x = L.Dropout(0.3)(x)

    ## 4.
    x = L.Dense(256)(x)
    x = L.LeakyReLU(0.2)(x)
    x = L.Dropout(0.3)(x)

    ## 5.
    x = L.Dense(1)(x)

    return Model(inputs, x, name="discriminator")

In [8]:
@tf.function
def train_step(real_images, latent_dim, generator, discriminator, g_opt, d_opt):
  batch_size = tf.cast(tf.shape(real_images)[0], tf.float32)
  bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)

  ## Discriminator
  batch_size = tf.cast(batch_size, tf.int32)
  latent_dim = tf.cast(latent_dim, tf.int32)
  noise = tf.random.normal([batch_size, latent_dim])

  for _ in range(2):
    with tf.GradientTape() as dtape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(real_images, training=True)
      fake_output = discriminator(generated_images, training=True)

      d_real_loss = bce_loss(tf.ones_like(real_output), real_output)
      d_fake_loss = bce_loss(tf.zeros_like(fake_output), fake_output)
      d_loss = d_real_loss + d_fake_loss

      d_grad = dtape.gradient(d_loss, discriminator.trainable_variables)
      d_opt.apply_gradients(zip(d_grad, discriminator.trainable_variables))


      ## Generator
      with tf.GradientTape() as gtape:
        generated_images = generator(noise, training=True)
        fake_output = discriminator(generated_images, training=True)

        g_loss = bce_loss(tf.ones_like(fake_output), fake_output)

        g_grad = gtape.gradient(g_loss, generator.trainable_variables)
        g_opt.apply_gradients(zip(g_grad, generator.trainable_variables))

        return d_loss, g_loss

In [9]:
def save_plot(examples, epoch, n):
  n = int(n)
  examples = (examples + 1) / 2.0
  examples = examples * 255
  file_name = f"samples/generated_plot_epoch-{epoch+1}.png"

  cat_image = None

  for i in range(n):
    start_idx = i*n
    end_idx = (i+1) * n
    image_list = examples[start_idx:end_idx]

    if i == 0:
      cat_image = np.concatenate(image_list, axis=1)

    else:
      tmp = np.concatenate(image_list, axis=1)
      cat_image = np.concatenate([cat_image, tmp], axis=0)

  cv2.imwrite(file_name, cat_image)

Hyperparameters

In [10]:
batch_size = 128
latent_dim = 64
num_epochs = 700
n_samples = 100

In [11]:
images_path = glob('images/*.jpg')
print(f"Images: {len(images_path)}")

Images: 19097


In [12]:
create_dir("Samples")
create_dir("Saved model")

Model

In [13]:
g_model = build_generator(latent_dim)
d_model = build_discriminator()

In [14]:
g_model.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 noise_input (InputLayer)    [(None, 64)]              0         
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1024)              263168    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 4096)              4198400   
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 4096)              0 

In [15]:
d_model.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 disc_input (InputLayer)     [(None, 64, 64, 3)]       0         
                                                                 
 flatten (Flatten)           (None, 12288)             0         
                                                                 
 dense_4 (Dense)             (None, 4096)              50335744  
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 4096)              0         
                                                                 
 dropout (Dropout)           (None, 4096)              0         
                                                                 
 dense_5 (Dense)             (None, 1024)              4195328   
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 1024)            

In [16]:
d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

In [17]:
images_dataset = tf_dataset(images_path, batch_size)
seed = np.random.normal(size=(n_samples, latent_dim))

In [18]:
images_dataset

<PrefetchDataset element_spec=TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None)>

In [19]:
for epoch in range(num_epochs):
        start = time.time()

        d_loss = 0.0
        g_loss = 0.0

        for image_batch in images_dataset:
            d_batch_loss, g_batch_loss = train_step(image_batch, latent_dim, g_model, d_model, g_optimizer, d_optimizer)
            d_loss += d_batch_loss
            g_loss += g_batch_loss

        d_loss = d_loss/len(images_dataset)
        g_loss = g_loss/len(images_dataset)

        g_model.save("saved_model/g_model.h5")
        d_model.save("saved_model/d_model.h5")

        examples = g_model.predict(seed, verbose=0)
        save_plot(examples, epoch, np.sqrt(n_samples))

        time_taken = time.time() - start
        print(f"[{epoch+1:1.0f}/{num_epochs}] {time_taken:2.2f}s - d_loss: {d_loss:1.4f} - g_loss: {g_loss:1.4f}")


[1/700] 6.71s - d_loss: 1.0993 - g_loss: 0.7549
[2/700] 4.73s - d_loss: 1.1207 - g_loss: 0.8228
[3/700] 4.73s - d_loss: 1.1795 - g_loss: 0.8890


: 